# Converting Loom to Seurat 
In this Jupyter Notebook, we explore how to import an example HCA cell by gene matrix in Loom format, explore it using the Bioconductor's AnVIL package, and convert it to a Seurat object that can be analyzed using the [Seurat Jupyter Notebook](https://app.terra.bio/#workspaces/featured-workspaces-hca/Intro-to-HCA-data-on-Terra/notebooks/launch/Seurat.ipynb).

This tutorial uses the uniform cell-by-gene count matrix from the HCA project ["Dissecting the human liver cellular landscape by single cell RNA-seq reveals novel intrahepatic monocyte/ macrophage populations"](https://data.humancellatlas.org/explore/projects/4d6f6c96-2a83-43d8-8fe1-0f53bffd4674).

Learn more about the data in "[Single cell RNA sequencing of human liver reveals distinct intrahepatic macrophage populations](https://www.nature.com/articles/s41467-018-06318-7)" (MacParland et al., 2018). 

## Notebook set-up

### Setting the Cloud Environment
This notebook uses the Terra R/Bioconductor base image (Python 3.7.10, R4.0.5) which you can select from the Cloud Environment widget at the top right of the Workspace. 

**Important! Please note the compute requirements for this notebook:**
- Select **4 CPUs** under the cloud compute profile in the Cloud Environment widget
- If you update the kernel after starting the notebook, delete the environment AND associated persistent disk before re-running the notebook

### Running the notebook
Prior to starting the notebook, we recommend restarting and clearing the kernel to remove outputs that were previously generated. This will help you understand what each code block is doing as the code is running. 

You can run the code blocks in this tutorial all at one time, or one-by-one. If you run each code block individually, make sure to run them in sequential order.

**Running all cells at one time**

To run all, select the "Cell" drop-down at the top of the notebook and choose "Run All".

**Running each cell individually**

You can run each cell individually in one of the following ways:
- Clicking the `Run` icon at the top of the notebook
- Using the shortcut shift return

## Installing notebook packages
Seurat is pre-installed on the R/Bioconductor base image in Terra. However, the code below installs additional Bioconductor and R packages (including the AnVIL and sceasy packages) that are useful for manipulating cloud files and for single-cell analysis.

#### What is the AnvIL package? 
The Bioconductor [AnVIL package](https://bioconductor.org/packages/release/bioc/html/AnVIL.html) is a suite of tools for working with Terra workspaces, data tables and external cloud resources, and includes utilities for movings files between cloud storage and the Terra Cloud Environment.

#### What is the sceasy package?
The sceasy package is a suite of tools for converting between file formats commonly used for single-cell analysis, such as Loom, AnnData (h5ad), and SingleCellExperiment (RDS). 

The code below installs the sceasy package following instructions in readme: https://github.com/cellgeni/sceasy. However, it does not use the conda option for installing anndata and loompy. Instead, anndata and loompy are installed using the `system()` command below.

In [1]:
# Installing sceasy-related packages
install.packages('reticulate')
system('pip install -U loompy', intern=FALSE)
system('pip install anndata', intern=FALSE)

# Installing Bioconductor packages
install_if_missing <- function(packages) {
    packages_needed <- setdiff(packages, rownames(installed.packages()))
    if (length(packages_needed) > 0) { 
        BiocManager::install(packages_needed)
    }
}

packages_for_this_notebook <- c(
    "AnVIL",
    "LoomExperiment",
    "SingleCellExperiment")

install_if_missing(packages_for_this_notebook)

library(AnVIL)
library(LoomExperiment)
library(reticulate)

Installing package into ‘/home/jupyter/notebooks/packages’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cloud.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.0 (2021-05-18)

Installing package(s) 'AnVIL', 'LoomExperiment'

also installing the dependencies ‘Rhdf5lib’, ‘rhdf5filters’, ‘rapiclient’, ‘rhdf5’, ‘HDF5Array’


Old packages: 'beachmat', 'bigrquery', 'BiocParallel', 'biomaRt', 'Biostrings',
  'blob', 'broom', 'cachem', 'cli', 'credentials', 'DelayedMatrixStats',
  'ensembldb', 'future', 'future.apply', 'GenomeInfoDb', 'GenomicFeatures',
  'gert', 'googlesheets4', 'haven', 'htmltools', 'httpuv', 'isoband', 'jpeg',
  'later', 'leiden', 'limma', 'MatrixGenerics', 'matrixStats', 'parallelly',
  'pillar', 'R6', 'RcppAnnoy', 'RcppArmadillo', 'RCurl', 'readr', 'reprex',
  'rmarkdown', 'RSQLite', 'rtracklayer', 'rvest', 'scuttle', 'Seurat',


In [2]:
# Installing sceasy package
devtools::install_github("cellgeni/sceasy")

RcppAnnoy    (0.0.18     -> 0.0.19    ) [CRAN]
utf8         (1.2.1      -> 1.2.2     ) [CRAN]
cli          (3.0.0      -> 3.0.1     ) [CRAN]
pillar       (1.6.1      -> 1.6.2     ) [CRAN]
spatstat.... (2.2-0      -> 2.2-2     ) [CRAN]
cachem       (1.0.5      -> 1.0.6     ) [CRAN]
later        (1.2.0      -> 1.3.0     ) [CRAN]
R6           (2.5.0      -> 2.5.1     ) [CRAN]
htmltools    (0.5.1.1    -> 0.5.2     ) [CRAN]
httpuv       (1.6.1      -> 1.6.2     ) [CRAN]
stringi      (1.6.2      -> 1.7.4     ) [CRAN]
tibble       (3.1.2      -> 3.1.4     ) [CRAN]
isoband      (0.2.4      -> 0.2.5     ) [CRAN]
parallelly   (1.26.1     -> 1.27.0    ) [CRAN]
RcppArmad... (0.10.5.0.0 -> 0.10.6.0.0) [CRAN]
matrixStats  (0.59.0     -> 0.60.1    ) [CRAN]
future       (1.21.0     -> 1.22.1    ) [CRAN]
future.apply (1.7.0      -> 1.8.1     ) [CRAN]
spatstat.... (2.2-0      -> 2.3-0     ) [CRAN]
leiden       (0.3.8      -> 0.3.9     ) [CRAN]
Seurat       (4.0.3      -> 4.0.4     ) [CRAN]


Skipping 2 packages not available: LoomExperiment, SingleCellExperiment

Installing 21 packages: RcppAnnoy, utf8, cli, pillar, spatstat.geom, cachem, later, R6, htmltools, httpuv, stringi, tibble, isoband, parallelly, RcppArmadillo, matrixStats, future, future.apply, spatstat.core, leiden, Seurat

Installing packages into ‘/home/jupyter/notebooks/packages’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘leiden’ had non-zero exit status”


✔  checking for file ‘/tmp/RtmpbfIVMK/remotes9b595fa293/cellgeni-sceasy-f8f0628/DESCRIPTION’
─  preparing ‘sceasy’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘sceasy_0.0.6.tar.gz’
   


Installing package into ‘/home/jupyter/notebooks/packages’
(as ‘lib’ is unspecified)



In [3]:
library(sceasy)

In [4]:
loompy <- reticulate::import('loompy')

## Installing environment variables
The following code uses AnVIL functions to assign several environment variables that allow you to access and manipulate Terra featues like the workspace name, Google bucket, and Google billing project. 

In [5]:
# Assign environment variables and view them
project <- avworkspace_namespace()
workspace <- avworkspace()
bucket <- avbucket()
project
workspace
bucket

[1] "featured-workspaces-hca"

[1] "featured-workspaces-hca/Intro-to-HCA-data-on-Terra-lk-20210901-notebook"

[1] "gs://fc-7324e65c-4d56-4a2e-973a-9c8f405bfce6"

## Examine file details for the HCA project matrix
HCA and other consortia data files live in cloud storage, like Google buckets, Amazon Web Services, and Azure. Since files can live in diverse storage options, they are often given a cloud agnostic GA4GH-compliant identifier called a DRS URI (read more in this Terra Support overview)

In this tutorial, we'll identify and move files between cloud environments using their DRS URIs. You can find an HCA data file's DRS URI using the AnVIL package:

1. Set the workspace containing the data using the avworkspace() function.
2. Examine the tables in the workspace using the avtable() function; this will return the participant table which Terra automatically creates when you import HCA data.
3. Set the participant table to a variable sample_set
4. Use dplyer to slice the table row containing the sample of interest; in this case, we want to combined project matrix (sc-landscape-human-liver-10XV2.loom) which is in the first row of the data table.
5. Use the pull command to pull the column containing DRS URI; in this case, it's the __loom__file_drs_uri column.

In [6]:
avworkspace("featured-workspaces-hca/Intro-to-HCA-data-on-Terra")
avtables()

fileName,size,contentType,gsUri,timeCreated,timeUpdated,bucket,name,googleServiceAccount,hashes
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<list>
sc-landscape-human-liver-10XV2.loom,1176122907,NA,gs://broad-datarepo-terra-prod-hca2-bucket/d30e68f8-c826-4639-88f3-ae35f00d4185/57130684-c89c-4b27-9030-298f50cbc95c/sc-landscape-human-liver-10XV2.loom,2021-02-19T19:31:35.668Z,2021-02-19T19:31:35.668Z,broad-datarepo-terra-prod-hca2-bucket,d30e68f8-c826-4639-88f3-ae35f00d4185/57130684-c89c-4b27-9030-298f50cbc95c/sc-landscape-human-liver-10XV2.loom,NULL,"b01eb717 , 9b990461e64e4c35fa67e466c4a22cba"


In [ ]:
sample_set <- avtable("participant")
sample_data <- sample_set %>% dplyr::slice(1)
View(sample_data)

In [ ]:
sample_name <- sample_data %>% pull("__loom__file_drs_uri")
sample_name

### Use the `drs_stat()` function to examine file details


- The AnVIL package provides the `drs_stat()` function as a useful way to look up files by DRS URIs and examine the the details before moving or manipulating the files.


- Before using the function, we'll assign it to a variable (`drs_url`) so that we can reuse the variable with other functions later in the notebook. 


- The table output below will show the file's name, size, date updated, and url for it's Google bucket location (in the "gsUri" column). 

In [ ]:
# Look at DRS URI info for the 10x project matrix :`sc-landscape-human-liver-10XV2.loom`
drs_url <- sample_name
stats <- drs_stat(drs_url)
stats

## Importing HCA data into the Cloud Environment
The AnVIL package uses the `drs_cp()` function to copy and move files with a DRS URI. 

The code below will automatically import the file into our virtual machine's [persistent disk](https://support.terra.bio/hc/en-us/articles/360047318551) file location. 

In [7]:
# Copy file with DRS URI into Cloud Envrionment
drs_cp(drs_url, ".")

fileName,size,contentType,gsUri,timeCreated,timeUpdated,bucket,name,googleServiceAccount,hashes,simple,destination
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<list>,<lgl>,<chr>
sc-landscape-human-liver-10XV2.loom,1176122907,NA,gs://broad-datarepo-terra-prod-hca2-bucket/d30e68f8-c826-4639-88f3-ae35f00d4185/57130684-c89c-4b27-9030-298f50cbc95c/sc-landscape-human-liver-10XV2.loom,2021-02-19T19:31:35.668Z,2021-02-19T19:31:35.668Z,broad-datarepo-terra-prod-hca2-bucket,d30e68f8-c826-4639-88f3-ae35f00d4185/57130684-c89c-4b27-9030-298f50cbc95c/sc-landscape-human-liver-10XV2.loom,NULL,"b01eb717 , 9b990461e64e4c35fa67e466c4a22cba",TRUE,./sc-landscape-human-liver-10XV2.loom


### Copying HCA data using gsutil
Since HCA data is stored in a Google bucket, we can also use Google's suite of tools for manipulating cloud data, [gsutil](https://cloud.google.com/storage/docs/gsutil). Forunately, the AnVIL package has gsutil built in. If you want to try it, just uncomment the code below. 

Same as above, this technique will import the file into our [persistent disk](https://support.terra.bio/hc/en-us/articles/360047318551) file location. 

We used the `drs_stat()` function above to find the Google bucket URI, but you can also find this information in the `participant` data table. When you click the project matrix DRS URI link in the table's `__loom__file_drs_uri` column, you'll notice a new window with a gsutil command for copying the file from a Google bucket location. This command contains the Google bucket URI. 

In [8]:
# home <- "/home/jupyter-user/notebooks/Intro-to-HCA-data-on-Terra-notebook-dev/edit"
# gs <- "gs://broad-datarepo-terra-prod-hca2-bucket/d30e68f8-c826-4639-88f3-ae35f00d4185/57130684-c89c-4b27-9030-298f50cbc95c/sc-landscape-human-liver-10XV2.loom"
# gsutil_cp(gs, home)

After importing the project matrix, we can confirm the import by listing files in our Cloud Environment's virtual machine. You should see the `sc-landscape-human-liver-10XV2.loom` file is now your Cloud Environment's file system.

In [9]:
# Confirm that `sc-landscape-human-liver-10XV2.loom` is in Cloud Environment disk
list.files()

[1] "Bioconductor.ipynb"                  "Pegasus.ipynb"                      
[3] "sc-landscape-human-liver-10XV2.loom" "Scanpy.ipynb"                       
[5] "Seurat.ipynb"

## Convert the Loom to AnnData with sceasy
To convert the HCA Loom to a Seurat object, we must first convert it to an intermediary AnnData object using the sceasy package.

The code below uses the sc-landscape-human-liver-10XV2.loom file as input and creates the output AnnData object which we'll store in an h5ad file called "trial_liver.h5ad."

In [10]:
# Convert to AnnData with sceasy; conversion takes a few minutes
sceasy::convertFormat('sc-landscape-human-liver-10XV2.loom', from="loom", to="anndata",
                       outFile='trial_liver.h5ad')

AnnData object with n_obs × n_vars = 332497 × 58347
    obs: 'cell_barcode_fraction_bases_above_30_mean', 'cell_barcode_fraction_bases_above_30_variance', 'cell_names', 'emptydrops_FDR', 'emptydrops_IsCell', 'emptydrops_Limited', 'emptydrops_LogProb', 'emptydrops_PValue', 'emptydrops_Total', 'fragments_per_molecule', 'fragments_with_single_read_evidence', 'genes_detected_multiple_observations', 'genomic_read_quality_mean', 'genomic_read_quality_variance', 'genomic_reads_fraction_bases_quality_above_30_mean', 'genomic_reads_fraction_bases_quality_above_30_variance', 'input_id', 'molecule_barcode_fraction_bases_above_30_mean', 'molecule_barcode_fraction_bases_above_30_variance', 'molecules_with_single_read_evidence', 'n_fragments', 'n_genes', 'n_mitochondrial_genes', 'n_mitochondrial_molecules', 'n_molecules', 'n_reads', 'pct_mitochondrial_molecules', 'perfect_cell_barcodes', 'perfect_molecule_barcodes', 'reads_mapped_exonic', 'reads_mapped_intergenic', 'reads_mapped_intronic', 'reads_ma

## Sanity check
Make sure the trial_liver.h5ad is now in the working directory by listing the files.

In [11]:
list.files()

[1] "Bioconductor.ipynb"                  "Pegasus.ipynb"                      
[3] "sc-landscape-human-liver-10XV2.loom" "Scanpy.ipynb"                       
[5] "Seurat.ipynb"                        "trial_liver.h5ad"

## Convert the AnnData to Seurat
A Seurat object is a container that holds count information from our count matrix, but also analysis results like PCA and clustering info.

Let's use the sceasy package to convert the trial_liver.h5ad to a Seurat object and write the object to an RDS file called "liver_seurat.rds".

In [12]:
# Use sceasy to convert AnnData to Seurat
sceasy::convertFormat('trial_liver.h5ad', from="anndata", to="seurat",
                       outFile='liver_seurat.rds')

Registered S3 method overwritten by 'spatstat.geom':
  method     from
  print.boxx cli 

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
X -> counts



An object of class Seurat 
58347 features across 332497 samples within 1 assay 
Active assay: RNA (58347 features, 0 variable features)

## Copy the Seurat object to the workspace Google bucket
Data generated by running an analysis in a Jupyter Notebook is saved to the disk associated with the Cloud Environment. When the disk is deleted, the data are as well. 

You can backup your work by moving files to your workspace Google bucket.


Remember those system environment variables set up at the beginning of this notebook? The code below uses those variables to simplify copying files to the workspace.


In [ ]:
# Copy the rds file generated in the notebook into the workspace bucket using AnVIL package gsutil commands
gsutil_cp("./*.rds", bucket)
# Run list command to see if file is in the bucket
gsutil_ls(bucket)

## Check the session info
The code below shows a record of all the packages used for this session.

In [ ]:
# Output all session information
devtools::session_info()

## Next steps
You can perform single-cell analysis of the Seurat object using the workspace [Seurat Jupyter Notebook](https://app.terra.bio/#workspaces/featured-workspaces-hca/Intro-to-HCA-data-on-Terra/notebooks/launch/Seurat.ipynb). Since some of the sceasy packages might conflict with the Seurat software, try **deleting your Cloud Environment and persistent disk** prior to using the Seurat Notebook.


## Congrats!
You've successfully completed the data import, exploration, and conversion tutorial for HCA data. Please leave feedback on the 
Terra [Featured Workspace Community Forum](https://support.terra.bio/hc/en-us/community/topics/360001603491). 